In [43]:
from nltk.corpus import stopwords
import numpy as np 
import string

In [21]:
corpus = open('/Users/ash/Downloads/pg1497.txt', 'r').readlines()

In [104]:
def clean_word(text): 
    for word in stopwords.words('english'): 
        text = text.replace(' '+word+' ',' ')
    puncs = ['-', ':', '.', ';', ',', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '(', ')', '?', '[', ']', '!', '/', "'"]
    for _ in puncs:
        text = text.replace(_,'')
    return text

def make_paragraphs(text):
    paragraphs = []
    dumb = ''
    for _ in range(len(text)): 
        if text[_] == '\n':
            paragraphs.append(dumb)
            dumb = ''
            continue
        else: 
            dumb += text[_]
            
    for i in range(len(paragraphs)):
        paragraphs[i] = paragraphs[i].replace('\n', ' ').lower()
        paragraphs[i] = clean_word(paragraphs[i])
    
    return paragraphs

paragraphs = make_paragraphs(corpus)
print(len(paragraphs))
new_paragraphs = []
for _ in range(len(paragraphs)): 
    if len(paragraphs[_]) > 30:
        new_paragraphs.append(paragraphs[_])
print(len(new_paragraphs))

4751
2878


In [105]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=100,
                                stop_words='english')
vectors = vectorizer.fit_transform(new_paragraphs)

In [106]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=15, max_iter=10,
                                learning_method='batch',
                                random_state=25)
lda.fit(vectors)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=15, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=25, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [107]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


names = vectorizer.get_feature_names()
print_top_words(lda, names, 10)

Topic #0: soul true like far city body things youth evil principle
Topic #1: state individual states better philosophy man like men greater education
Topic #2: men gods god women let say like things best man
Topic #3: state rulers guardians children citizens replied common city best way
Topic #4: life great man human world men nature away good time
Topic #5: mind say suppose light right body thought sort sense answer
Topic #6: justice injustice unjust virtue socrates man say glaucon nature argument
Topic #7: man shall let saying nature think say true answer make
Topic #8: manner end argument words republic plato make laws socrates state
Topic #9: art idea modern plato truth ideas greatest knowledge mind like
Topic #10: said yes thing ought say sort able mean glaucon think
Topic #11: plato number greek education use order form law natural common
Topic #12: reason pleasure pleasures second come like true form truth principle
Topic #13: good knowledge evil opinion mean like know truth man

In [10]:
import numpy as np
from collections import Counter

stick_breaks = np.asarray([])


def sample_stick_breaking_process(stick_breaks):
    s = np.random.rand()
    lst = np.where(s < stick_breaks)[0]
    if lst.size:
        return (stick_breaks, lst[0])
    else:
        # We have sampled a new category!
        # Break the stick in a new place and update the array!
        if stick_breaks.size == 0:
            start = 0
        else:
            start = stick_breaks[-1]
        width = 1.0 - start
        pos = start + width * np.random.rand()
        stick_breaks = np.append(stick_breaks, [pos])
        return (stick_breaks, len(stick_breaks))


N = 10000
samples = np.zeros(N)
for n in range(N):
    (stick_breaks, samples[n]) = sample_stick_breaking_process(stick_breaks)

print(stick_breaks)
print(Counter(samples))

[0.68681757 0.97639165 0.99565539 0.99707382 0.99744252 0.99755216
 0.9984785  0.99910555 0.99982863 0.99995316]
Counter({0.0: 6828, 1.0: 2931, 2.0: 196, 8.0: 12, 3.0: 11, 7.0: 8, 4.0: 5, 6.0: 5, 5.0: 2, 9.0: 1, 10.0: 1})
